In [1]:
import pandas as pd
import numpy as np
from datasets import Dataset
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix
import time
from transformers import TrainerCallback
import torch
from transformers import pipeline
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer
)

2025-11-29 21:56:03.782337: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764453363.963778      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764453364.016281      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

rubert-base-cased-conversational
Conversational RuBERT (Russian, cased, 12‑layer, 768‑hidden, 12‑heads, 180M parameters) was trained on OpenSubtitles[1], Dirty, Pikabu, and a Social Media segment of Taiga corpus[2]. We assembled a new vocabulary for Conversational RuBERT model on this data and initialized the model with RuBERT.

08.11.2021: upload model with MLM and NSP heads

[1]: P. Lison and J. Tiedemann, 2016, OpenSubtitles2016: Extracting Large Parallel Corpora from Movie and TV Subtitles. In Proceedings of the 10th International Conference on Language Resources and Evaluation (LREC 2016)

[2]: Shavrina T., Shapovalova O. (2017) TO THE METHODOLOGY OF CORPUS CONSTRUCTION FOR MACHINE LEARNING: «TAIGA» SYNTAX TREE CORPUS AND PARSER. in proc. of “CORPORA2017”, international conference , Saint-Petersbourg, 2017.

In [2]:
# =========================
# 1. ЗАГРУЗКА ДАННЫХ
# =========================
df = pd.read_csv("/kaggle/input/hack-data/train.csv")  


df = df.sample(frac=1).reset_index(drop=True)  # shuffle

In [4]:
train_df, temp_df = train_test_split(
    df,
    test_size=0.2,  
    stratify=df["label"],
    random_state=42
)

val_df, test_df = train_test_split(
    temp_df,
    test_size=0.5,              
    stratify=temp_df["label"], 
    random_state=42
)

train_ds = Dataset.from_pandas(train_df)
val_ds = Dataset.from_pandas(val_df)
test_ds = Dataset.from_pandas(test_df)

rubert-base-cased-conversational Conversational RuBERT (Russian, cased, 12‑layer, 768‑hidden, 12‑heads, 180M parameters) was trained on OpenSubtitles[1], Dirty, Pikabu, and a Social Media segment of Taiga corpus[2]. We assembled a new vocabulary for Conversational RuBERT model on this data and initialized the model with RuBERT.

08.11.2021: upload model with MLM and NSP heads

[1]: P. Lison and J. Tiedemann, 2016, OpenSubtitles2016: Extracting Large Parallel Corpora from Movie and TV Subtitles. In Proceedings of the 10th International Conference on Language Resources and Evaluation (LREC 2016)

[2]: Shavrina T., Shapovalova O. (2017) TO THE METHODOLOGY OF CORPUS CONSTRUCTION FOR MACHINE LEARNING: «TAIGA» SYNTAX TREE CORPUS AND PARSER. in proc. of “CORPORA2017”, international conference , Saint-Petersbourg, 2017.

In [5]:
MODEL_NAME = "DeepPavlov/rubert-base-cased-conversational"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def tokenize(batch):
    return tokenizer(
        batch["text"],
        padding="max_length",
        truncation=True,
        max_length=256
    )

train_ds = train_ds.map(tokenize, batched=True)
val_ds = val_ds.map(tokenize, batched=True)

train_ds = train_ds.remove_columns(["text"])
val_ds = val_ds.remove_columns(["text"])

train_ds.set_format("torch")
val_ds.set_format("torch")

tokenizer_config.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Map:   0%|          | 0/185892 [00:00<?, ? examples/s]

Map:   0%|          | 0/23237 [00:00<?, ? examples/s]

In [6]:
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=3  # 0,1,2
)

pytorch_model.bin:   0%|          | 0.00/714M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased-conversational and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)

    return {
        "accuracy": accuracy_score(labels, preds),
        "f1_macro": f1_score(labels, preds, average="macro")
    }

In [ ]:
# =========================
# 5. ПАРАМЕТРЫ ОБУЧЕНИЯ
# =========================
# training_args = TrainingArguments(
#     output_dir="/kaggle/working/rubert_sentiment",
#     eval_strategy="epoch",
#     save_strategy="epoch",
#     learning_rate=2e-5,
#     per_device_train_batch_size=16,
#     per_device_eval_batch_size=16,
#     num_train_epochs=1,
#     weight_decay=0.01,
#     logging_dir="./logs",
#     load_best_model_at_end=True,
#     metric_for_best_model="f1_macro",
    
# )

In [12]:
# =========================
# 5. ПАРАМЕТРЫ ОБУЧЕНИЯ
# =========================
training_args = TrainingArguments(
    output_dir="/kaggle/working/rubert_sentiment",
    eval_strategy="steps",
    eval_steps=500,
    
    save_strategy="steps",
    save_steps=1000,
    
    logging_strategy="steps",
    logging_steps=1000,
    logging_first_step=True,
    
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=1,
    weight_decay=0.01,
    metric_for_best_model="f1_macro",

    save_total_limit=1,
    save_only_model=True,
    load_best_model_at_end=True,
    no_cuda=False,
    
    logging_dir=None,
    report_to="none",
    
    dataloader_pin_memory=True,
    dataloader_num_workers=2,
)

In [13]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/tmp/ipykernel_47/1092916359.py:5: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [21]:
class ProgressCheckerCallback(TrainerCallback):
    def on_step_begin(self, args, state, control, **kwargs):
        if state.global_step % 1000 == 0:
            print(f"🔹 Прогресс: шаг {state.global_step}, эпоха {state.epoch:.1f}", flush=True)
    
    def on_train_begin(self, args, state, control, **kwargs):
        print("Обучение НАЧАЛОСЬ!", flush=True)
    
    def on_step_end(self, args, state, control, **kwargs):
        # Гарантированный вывод каждые 5 секунд
        if hasattr(self, 'last_print'):
            if time.time() - self.last_print > 3600:
                print(f"Активно обучается... шаг {state.global_step}", flush=True)
                self.last_print = time.time()
        else:
            self.last_print = time.time()

In [22]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[ProgressCheckerCallback()]
)

/tmp/ipykernel_47/1212326759.py:5: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [23]:
trainer.train()

🎯 Обучение НАЧАЛОСЬ!
🔹 Прогресс: шаг 0, эпоха 0.0


Step,Training Loss,Validation Loss,Accuracy,F1 Macro
500,1.152100,0.603310,0.733098,0.731142
1000,0.635400,0.582738,0.743039,0.741786
1500,0.635400,0.568565,0.751861,0.752625
2000,0.575800,0.559749,0.751646,0.749541
2500,0.575800,0.554814,0.752808,0.751764
3000,0.558000,0.543925,0.755907,0.757262
3500,0.558000,0.543965,0.758704,0.760996
4000,0.548700,0.533690,0.762491,0.765050
4500,0.548700,0.531868,0.767354,0.768245
5000,0.531500,0.525866,0.768903,0.767910


🔹 Прогресс: шаг 1000, эпоха 0.2
🔹 Прогресс: шаг 2000, эпоха 0.3
🔹 Прогресс: шаг 3000, эпоха 0.5
⏱️  Активно обучается... шаг 3001
🔹 Прогресс: шаг 4000, эпоха 0.7
🔹 Прогресс: шаг 5000, эпоха 0.9


TrainOutput(global_step=5810, training_loss=0.5645842586450035, metrics={'train_runtime': 6972.5437, 'train_samples_per_second': 26.661, 'train_steps_per_second': 0.833, 'total_flos': 2.445533972414669e+16, 'train_loss': 0.5645842586450035, 'epoch': 1.0})

In [ ]:
trainer.save_model()

In [19]:
# метрики предобученной модельки
model = AutoModelForSequenceClassification.from_pretrained("/kaggle/input/my-model/rubert_sentiment_v2")
tokenizer = AutoTokenizer.from_pretrained("/kaggle/input/my-model/rubert_sentiment_v2")


texts = test_df["text"].astype(str).tolist()
true_labels = test_df["label"].astype(int).tolist()

device = 0 if torch.cuda.is_available() else -1

classifier = pipeline(
    "sentiment-analysis",
    model=model,
    tokenizer=tokenizer,
    device=device,
    max_length=512
)

label_map = {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
}

pred_labels = []
for text in tqdm(texts):
    result = classifier(text, truncation=True)[0]["label"]
    pred_labels.append(label_map[result])

acc = accuracy_score(true_labels, pred_labels)
f1_macro = f1_score(true_labels, pred_labels, average="macro")

print("\n✅ RESULTS")
print(f"Accuracy:  {acc:.4f}")
print(f"F1-macro:  {f1_macro:.4f}")

print("\n📊 CLASSIFICATION REPORT:")
print(classification_report(true_labels, pred_labels, target_names=["NEGATIVE", "NEUTRAL", "POSITIVE"]))

print("\n🧮 CONFUSION MATRIX:")
print(confusion_matrix(true_labels, pred_labels))

Device set to use cuda:0
100%|██████████| 23237/23237 [03:26<00:00, 112.79it/s]



✅ RESULTS
Accuracy:  0.7997
F1-macro:  0.7996

📊 CLASSIFICATION REPORT:
              precision    recall  f1-score   support

    NEGATIVE       0.71      0.72      0.72      7725
     NEUTRAL       0.88      0.89      0.89      7749
    POSITIVE       0.80      0.79      0.80      7763

    accuracy                           0.80     23237
   macro avg       0.80      0.80      0.80     23237
weighted avg       0.80      0.80      0.80     23237


🧮 CONFUSION MATRIX:
[[5557  782 1386]
 [ 741 6879  129]
 [1487  130 6146]]


# **Попытка прогона готового решения**

In [14]:
import pandas as pd
import torch
from transformers import pipeline
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix
from tqdm import tqdm

In [18]:
# ============================
# 1. ЗАГРУЖАЕМ ДАННЫЕ
# ============================

# df = pd.read_csv("/kaggle/input/hack-data/train.csv")
# texts = test_df["text"].astype(str).tolist()
# true_labels = test_df["label"].astype(int).tolist()

In [15]:
# ============================
# 2. СОЗДАЁМ PIPELINE
# ============================

MODEL_NAME = "blanchefort/rubert-base-cased-sentiment"

device = 0 if torch.cuda.is_available() else -1

blanch_classifier = pipeline(
    "sentiment-analysis",
    model=MODEL_NAME,
    tokenizer=MODEL_NAME,
    device=device
)

config.json:   0%|          | 0.00/943 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/711M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/499 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cuda:0


In [19]:
# ============================
# 3. ПРОГОН МОДЕЛИ
# ============================
blanch_pred_labels = []

blanch_label_map = {
    "NEGATIVE": 0,
    "NEUTRAL": 1,
    "POSITIVE": 2
}

print("Запуск инференса...")

for text in tqdm(texts):
    result = blanch_classifier(text, truncation=True)[0]["label"]
    blanch_pred_labels.append(blanch_label_map[result])

Запуск инференса...


100%|██████████| 23237/23237 [03:49<00:00, 101.13it/s]


In [20]:
acc = accuracy_score(true_labels, blanch_pred_labels)
f1_macro = f1_score(true_labels, blanch_pred_labels, average="macro")

print("\n✅ RESULTS")
print(f"Accuracy:  {acc:.4f}")
print(f"F1-macro:  {f1_macro:.4f}")

print("\n📊 CLASSIFICATION REPORT:")
print(classification_report(true_labels, blanch_pred_labels, target_names=["NEGATIVE", "NEUTRAL", "POSITIVE"]))

print("\n🧮 CONFUSION MATRIX:")
print(confusion_matrix(true_labels, blanch_pred_labels))



✅ RESULTS
Accuracy:  0.1874
F1-macro:  0.1847

📊 CLASSIFICATION REPORT:
              precision    recall  f1-score   support

    NEGATIVE       0.29      0.33      0.31      7725
     NEUTRAL       0.20      0.16      0.18      7749
    POSITIVE       0.07      0.07      0.07      7763

    accuracy                           0.19     23237
   macro avg       0.18      0.19      0.18     23237
weighted avg       0.18      0.19      0.18     23237


🧮 CONFUSION MATRIX:
[[2584 3412 1729]
 [ 749 1247 5753]
 [5638 1601  524]]
